In [114]:
import pandas as pd
import numpy as np

In [115]:
df = pd.read_csv("./data/Section 1 data.csv")

In [116]:
#Question 1 
unique_businesses = df["Business_ID"].nunique()
print("\nQ1) Number of unique businesses:", unique_businesses)


Q1) Number of unique businesses: 30276


In [117]:
#Question 2 
total_reviews = len(df)
reviews_by_business = df.groupby(["Business_ID","Business_Name"],dropna=False,as_index=False).size().rename(columns={"size":"Review_Count"})
top_row = reviews_by_business.sort_values(by="Review_Count",ascending=False).iloc[0]
top_business_id = top_row["Business_ID"]
top_name = top_row["Business_Name"]
top_count = int(top_row["Review_Count"])
top_percent = top_count/total_reviews *100.0
print("\nQ2) Most reviewed restaurant:")
print(f"    Business_ID: {top_business_id}")
print(f"    Name: {top_name}")
print(f"    # Reviews: {top_count}")
print(f"    % of all reviews: {top_percent:.2f}%")


Q2) Most reviewed restaurant:
    Business_ID: 4bEjOyTaDG24SY5TxsaUNQ
    Name: Mon Ami Gabi
    # Reviews: 856
    % of all reviews: 0.38%


In [118]:
#Question 3 
nevada_df = df[(df["State"].str.upper() == "NV") & (df["Avg_Business_Star_Rating"] == 5.0)]
unique_nevada_cities = sorted(nevada_df["City"].unique().tolist())
print("\nQ3) NV cities with at least one 5-star review (Avg_Business_Star_Rating == 5):")
print("   ", unique_nevada_cities if unique_nevada_cities else "None found")


Q3) NV cities with at least one 5-star review (Avg_Business_Star_Rating == 5):
    ['Boulder City', 'Henderson', 'Las Vegas', 'Nellis']


In [119]:
#Question 4 
hotels_travel_df = df[df["Business_Category"].astype(str).str.strip().str.lower() == "hotels & travel"]
total_reviews_hotels = len(hotels_travel_df)
if total_reviews_hotels == 0:
    print("    No rows found for Business_Category == 'Hotels & Travel' (after normalization).")
else:
    hotels_travels_by_business = hotels_travel_df.groupby(["City","State"],dropna=False,as_index=False).size().rename(columns={"size":"Review_Count"})
    top_city_df = hotels_travels_by_business.sort_values(by="Review_Count",ascending=False).iloc[0]
    top_city_hotels = top_city_df["City"]
    top_state_hotels = top_city_df["State"]
    top_review_count_hotels = top_city_df["Review_Count"]
    top_city_pct = 100* top_review_count_hotels / total_reviews_hotels
    print("\nQ4) Hotels & Travel — city with most reviews:")
    print(f"    City: {top_city_hotels}")
    print(f"    State: {top_state_hotels}")
    print(f"    # Reviews: {top_review_count_hotels}")
    print(f"    % of Hotels & Travel reviews: {top_city_pct:.2f}%")



Q4) Hotels & Travel — city with most reviews:
    City: Las Vegas
    State: NV
    # Reviews: 10245
    % of Hotels & Travel reviews: 77.64%


In [120]:
#Question 5
df["Review_Date"] = pd.to_datetime(df["Review_Date"],errors='coerce')
df["DayOfWeek"] = df["Review_Date"].dt.day_name()
day_grouped = df.groupby("DayOfWeek",as_index=False,dropna=False).size().rename(columns={"size":"Review_Count"})
top_dow_df = day_grouped.sort_values(by="Review_Count",ascending=False).iloc[0]
top_dow = top_dow_df["DayOfWeek"]
top_dow_count = top_dow_df["Review_Count"]
top_dow_pct = 100.0 * top_dow_count / total_reviews
print("\nQ5) Most common review posting day:")
print(f"    Day: {top_dow}")
print(f"    # Reviews: {top_dow_count}")
print(f"    % of all reviews: {top_dow_pct:.2f}%")


Q5) Most common review posting day:
    Day: Monday
    # Reviews: 36446
    % of all reviews: 16.01%


In [121]:
#Question 6
#(A) Overall monthly trend (volume + average rating)
df["YearMonth"] = df["Review_Date"].dt.to_period("M")

monthly_trend = (
    df.groupby("YearMonth", dropna=False)
      .agg(
          reviews=("Business_ID", "size"),
          avg_rating=("Avg_Business_Star_Rating", "mean"),
          median_rating=("Avg_Business_Star_Rating", "median"),
      )
      .reset_index()
      .sort_values("YearMonth")
)

monthly_trend["YearMonth"] = monthly_trend["YearMonth"].astype(str)
print(monthly_trend)

    YearMonth  reviews  avg_rating  median_rating
0     2005-01        1    3.000000            3.0
1     2005-03       10    3.800000            4.0
2     2005-04        6    3.833333            4.0
3     2005-05        7    3.714286            3.5
4     2005-06        1    4.000000            4.0
..        ...      ...         ...            ...
109   2014-03     6856    3.790986            4.0
110   2014-04     6654    3.794034            4.0
111   2014-05     6730    3.805423            4.0
112   2014-06     6742    3.799614            4.0
113   2014-07     4168    3.787548            4.0

[114 rows x 4 columns]


In [122]:
#Q6
#(B) Per-business monthly trend
business_monthly = (
    df.groupby(["Business_ID", "Business_Name", "YearMonth"], dropna=False)
      .agg(
          monthly_reviews=("Business_ID", "size"),
          avg_rating=("Avg_Business_Star_Rating", "mean"),
      )
      .reset_index()
      .sort_values(["Business_ID", "YearMonth"])
)

business_monthly["YearMonth"] = business_monthly["YearMonth"].astype(str)
business_monthly = business_monthly.sort_values("YearMonth", ascending=True)
print(business_monthly.head())

                   Business_ID                   Business_Name YearMonth  \
100631  bYhpy9u8fKkGhYHtvYXazQ  Paris Las Vegas Hotel & Casino   2005-01   
157327  y7GRFf2jsQxUwu1pw5kkXQ         Edo Japanese Restaurant   2005-03   
79056   TypkaxMUTthOfl_Btj5X1A              Glass Nickel Pizza   2005-03   
86598   WnY4HPJIYNXOPQH2mFzl2Q        THEhotel at Mandalay Bay   2005-03   
73544   RgBq9TFI8q6-vCvF6wOMVg                  Genna's Lounge   2005-03   

        monthly_reviews  avg_rating  
100631                1         3.0  
157327                1         3.0  
79056                 1         3.5  
86598                 1         4.0  
73544                 1         4.0  


Q6 – Trend Summary

Review volume increases significantly over time, indicating growing customer engagement and platform expansion. At the same time, the average restaurant rating gradually improves and stabilizes around 3.7–3.8. The relatively narrow range of ratings suggests consistent evaluation behavior and increasing marketplace maturity. Overall, the data indicates both growth in activity and modest improvement in perceived restaurant quality over time.

In [123]:
#Q7
business_stats = (
    df.groupby(["Business_ID", "Business_Name"], dropna=False)
      .agg(
          review_count=("Business_ID", "size"),
          avg_rating=("Avg_Business_Star_Rating", "mean"),
      )
      .reset_index()
)

volume_threshold = business_stats["review_count"].quantile(0.90)  
rating_median = business_stats["avg_rating"].median()

improvement_targets = business_stats[
    (business_stats["review_count"] >= volume_threshold) &
    (business_stats["avg_rating"] < rating_median)
].sort_values(["avg_rating", "review_count"], ascending=[True, False])

print(improvement_targets.head(20))

                  Business_ID                            Business_Name  \
8536   GhQQsSApIRfUmIcqzmSaGA                          Spirit Airlines   
22190  inDYSqSDMyrfBhOZc702rw                       Cox Communications   
10378  KVYSUfN89b-nih-_9HhF6g           McFadden's Restaurant & Saloon   
7341   EF5vR_UUavzPjtgMYFCg5Q                      Carlos' n Charlie's   
28825  wyqwO4ZqZyRR6pGk1pTBrA                       Payless Car Rental   
20950  g2VN-L5UPk5haGVeSe1LEg                       Payless Car Rental   
10191  K5kU2IN6mXvMo-Cx0oQFZg                            Calico Jack's   
25418  pjtB4FadDdq_DeyedjDsOw         Danny's Family Carousel Car Wash   
25490  ptcXvjH3L4buWCI2J9LAQA                                   OnTrac   
12861  PdZj1gylTC5AaTO21eOYcA                                   Ah Sin   
1533   26ecLqpDHrB3socZGfDuhQ                           Showtime Tours   
6549   Ca_u-CVUEswSkdmW0oscvA                        Dollar Rent a Car   
6407   CJ8IURVsHxTIu2Kzp2Kumw         

In [124]:
#Q7
targets_ids = improvement_targets["Business_ID"]

target_reviews = df[df["Business_ID"].isin(targets_ids)].copy()
target_reviews["Review_Text"] = target_reviews["Review_Text"].astype(str).str.lower()

keywords = ["slow", "rude", "cold", "dirty", "wait", "overpriced", "bad", "terrible", "bland"]

results = []

for k in keywords:
    temp = (
        target_reviews[target_reviews["Review_Text"].str.contains(rf"\b{k}\b", regex=True)]
        .groupby(["Business_ID", "Business_Name"], dropna=False)
        .size()
        .reset_index(name="count")
    )
    temp["keyword"] = k
    results.append(temp)

keyword_business_df = pd.concat(results, ignore_index=True)

# Sort nicely
keyword_business_df = keyword_business_df.sort_values(
    ["Business_ID", "count"], ascending=[True, False]
)
pivot_df = (
    keyword_business_df
    .pivot_table(
        index=["Business_ID", "Business_Name"],
        columns="keyword",
        values="count",
        fill_value=0
    )
    .reset_index()
)

print(pivot_df.head())


keyword             Business_ID    Business_Name  bad  bland  cold  dirty  \
0        -36CwMJZJigqPMYrx9BJzw     TGI Friday's  3.0    0.0   2.0    1.0   
1        -9pVS__IliMA2aNEYzrQrg  American Junkie  2.0    0.0   1.0    1.0   
2        -O7H5LhBkE13VQZB0QZI0Q      Fat Tuesday  3.0    0.0   0.0    2.0   
3        -VnVvJzps8QkDbc1GmHnmQ      Studio Café  6.0    1.0   3.0    0.0   
4        -WZIxGXJHMGidZXRhKxP3w      Mimi's Cafe  1.0    0.0   0.0    0.0   

keyword  overpriced  rude  slow  terrible  wait  
0               0.0   0.0   2.0       3.0   1.0  
1               0.0   1.0   2.0       1.0   3.0  
2               0.0   0.0   0.0       0.0   0.0  
3               5.0   0.0   5.0       2.0   4.0  
4               0.0   0.0   1.0       1.0   1.0  


Q7 – Recommendations to Improve Public Appeal

Based on the analysis, several businesses show high review volumes but below-median average ratings. This suggests strong visibility but potential quality or service issues.

1. Improve Service Efficiency

Frequent mentions of keywords such as “slow” and “wait” indicate service delays. Restaurants should:

Optimize staffing during peak hours

Improve kitchen workflow efficiency

Reduce customer waiting time

2. Enhance Staff Training and Customer Experience

Keywords such as “rude” and “bad” suggest issues related to service behavior. Restaurants should:

Provide customer service training

Monitor and respond to customer complaints

Improve front-of-house communication

3. Improve Food Quality and Consistency

Mentions of “cold” and “bland” indicate potential quality control problems. Restaurants should:

Ensure consistent food preparation standards

Improve quality checks before serving

In [125]:
geo = df.dropna(subset=["Latitude", "Longitude"]).copy()

geo = geo.sort_values(["User_ID", "Review_Date"])

geo["prev_lat"] = geo.groupby("User_ID")["Latitude"].shift(1)
geo["prev_lon"] = geo.groupby("User_ID")["Longitude"].shift(1)

moves = geo.dropna(subset=["prev_lat", "prev_lon"]).copy()

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

moves["distance_km"] = haversine_km(
    moves["prev_lat"].to_numpy(),
    moves["prev_lon"].to_numpy(),
    moves["Latitude"].to_numpy(),
    moves["Longitude"].to_numpy(),
)
user_distance = (
    moves.groupby("User_ID", dropna=False)["distance_km"]
         .sum()
         .reset_index()
         .sort_values("distance_km", ascending=False)
)

top_user = user_distance.iloc[0]
print("Top traveler user:", top_user["User_ID"])
print("Distance (km):", round(top_user["distance_km"], 2))
print("Users analyzed:", len(user_distance))

Top traveler user: 6uYJ-ixRxPMyf-iEbhoz2g
Distance (km): 31766.8
Users analyzed: 34588
